<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Data scrape

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [15]:
# Imports
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = urlopen(url).read().decode('utf-8')


soup = BeautifulSoup(html_doc, 'html.parser')

toronto_table = soup.body.find(id="mw-content-text").find("table").tbody

#print(toronto_table.prettify())

In [17]:
table_data = []
i = 0
for table_row in toronto_table.find_all('tr'):    
    if (i==0):
        i+=1
        continue
    #print("Row ",i)
    row = []
    for cell in table_row.find_all('td'):
        row.append(cell.text.strip())
    table_data.append(row)
    i+=1

In [18]:
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(table_data, columns=columns)

In [19]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Cleanup
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [20]:
df = df[(df["Borough"] != "Not assigned")]

In [31]:
df["Neighbourhood"].replace("Not assigned", df["Borough"], inplace=True)

In [34]:
df['Neighbourhood'] = df.groupby('Postcode')['Neighbourhood'].transform(lambda x: "%s" % ', '.join(x)).values
df = df.drop_duplicates().reset_index(drop=True)

In [33]:
#df[df["Neighbourhood"] == "Not assigned"]

In [35]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [36]:
df.shape

(103, 3)

In [37]:
df.to_csv("toronto_data.csv")